<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/btp_using_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')

In [3]:
d_tweets=d_tweets_dataset['tweet']

In [4]:
non_d_tweets=non_d_tweets_dataset['tweet']

In [5]:
y1=np.ones((3082,1),dtype="int")

In [6]:
y2=np.zeros((2505,1),dtype="int")

In [7]:
y=np.concatenate((y1,y2),axis=0)

In [8]:
res=pd.DataFrame(y)

In [9]:
txt=pd.concat([d_tweets,non_d_tweets],axis=0)

In [10]:
txt.shape
txt=txt.reset_index(drop="true")

In [11]:
df=pd.concat([txt,res],axis=1,join="inner")

In [12]:
df.columns=["text","result"]


In [13]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]

In [15]:
from gensim.models import FastText

# Train FastText model
fasttext_model = FastText(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [16]:
# Get the FastText word vectors
word_vectors = fasttext_model.wv


In [17]:
# Convert text to FastText word embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)
X_sequences = tokenizer.texts_to_sequences(tokenized_text)
X_padded = pad_sequences(X_sequences, maxlen=100)

# X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100)
# X_fasttext_padded
# y = df['result'].values


In [18]:
# Prepare target variable y (same as before)

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [19]:
X_fasttext = []
for sentence in tokenized_text:
    embedding = []
    for word in sentence:
        if word in word_vectors:
            embedding.append(word_vectors[word])
    X_fasttext.append(embedding)

# Find the maximum sequence length in X_fasttext
max_sequence_length = max(len(embedding) for embedding in X_fasttext)

# Pad sequences to the maximum length
X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100, dtype='float32', padding='post')
X_fasttext_padded.shape
y = df['result'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_fasttext_padded, y, test_size=0.2, random_state=42)

In [21]:
# Define the LSTM-RNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(100, 100)))

# Add more layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Dropout layer for regularization

# # Add another LSTM layer with 64 units, dropout, and recurrent dropout
# model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))

# # Add more layers
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))  # Dropout layer for regularization

# Output layer with a single unit and sigmoid activation function for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Step 7: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 8: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')
# Step 9: Make Predictions (same as before)


Epoch 1/10
  8/140 [>.............................] - ETA: 52s - loss: 0.6918 - accuracy: 0.5234

KeyboardInterrupt: ignored

Using Glove

In [23]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


OSError: ignored

In [24]:
!pip install spacy

In [25]:
!python -m spacy download en_core_web_md

2023-11-13 11:32:13.871726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 11:32:13.871818: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 11:32:13.871870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 11:32:15.566794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [26]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


In [27]:
# Convert text to GloVe word embeddings
X_glove = []
for sentence in tokenized_text:
    embedding = [get_word_vector(word) for word in sentence]
    embedding = [word_embedding for word_embedding in embedding if word_embedding is not None]
    X_glove.append(embedding)

# Find the maximum sequence length in X_glove
max_sequence_length = max(len(embedding) for embedding in X_glove)

# Pad sequences to the maximum length
X_glove_padded = pad_sequences(X_glove, maxlen=max_sequence_length, dtype='float32', padding='post')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_glove_padded, y, test_size=0.2, random_state=42)


In [29]:
# Define the LSTM-RNN Model
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(max_sequence_length, 300)))  # 300 is the dimensionality of GloVe vectors
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# # Step 6: Make Predictions
# predictions = model.predict_classes(X_test)


Epoch 1/10


KeyboardInterrupt: ignored

Using TF-IDF Vectorization

In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [31]:
X = df['text']
y = df['result']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [33]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [35]:
vocab_size = len(tokenizer.word_index) + 1
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_test_seq))
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(LSTM(128))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.1)


In [ ]:
y_pred_probs = model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int)  # Apply thresholding at 0.5 for binary classification

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Using BiLSTM and word2vec

In [42]:
import pandas as pd
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [38]:
max_words = 10000  # Choose based on your dataset
maxlen = 100

In [43]:

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

In [40]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 6236 unique tokens.


In [41]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

In [44]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

Found 6236 unique tokens.


In [45]:
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [52]:
embedding_dim = 100  # Should match the vector_size in Word2Vec model

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))


In [53]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred = model.predict_classes(x_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Epoch 1/10
140/140 [==============================] - 36s 188ms/step - loss: 0.4552 - accuracy: 0.7776 - val_loss: 0.3082 - val_accuracy: 0.8605
Epoch 2/10
140/140 [==============================] - 23s 164ms/step - loss: 0.1970 - accuracy: 0.9224 - val_loss: 0.2939 - val_accuracy: 0.8757
Epoch 3/10
140/140 [==============================] - 25s 178ms/step - loss: 0.1098 - accuracy: 0.9595 - val_loss: 0.3399 - val_accuracy: 0.8784
Epoch 4/10
140/140 [==============================] - 29s 207ms/step - loss: 0.0664 - accuracy: 0.9752 - val_loss: 0.3848 - val_accuracy: 0.8873
Epoch 5/10
140/140 [==============================] - 27s 189ms/step - loss: 0.0457 - accuracy: 0.9828 - val_loss: 0.4635 - val_accuracy: 0.8846
Epoch 6/10
140/140 [==============================] - 22s 155ms/step - loss: 0.0346 - accuracy: 0.9877 - val_loss: 0.5085 - val_accuracy: 0.8927
Epoch 7/10
140/140 [==============================] - 25s 178ms/step - loss: 0.0296 - accuracy: 0.9886 - val_loss: 0.4992 - val_ac

AttributeError: ignored

Using BiLSTM and fasttext

In [55]:
import pandas as pd
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
# Assuming you have a DataFrame named 'df' with 'text' and 'result' columns
# df = pd.read_csv('your_dataset.csv')

# Tokenize the text
max_words = 10000  # Choose based on your dataset
maxlen = 100  # Choose based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

data = pad_sequences(sequences, maxlen=maxlen)
labels = df['result'].values

# Train your FastText model
sentences = [text.split() for text in df['text']]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Build the BiLSTM model
embedding_dim = 100  # Should match the vector_size in FastText model

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add((Dense(64,activation="relu")))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
y_pred = model.predict_classes(x_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

Found 6236 unique tokens.
Epoch 1/10
140/140 [==============================] - 32s 180ms/step - loss: 0.4612 - accuracy: 0.7644 - val_loss: 0.2984 - val_accuracy: 0.8515
Epoch 2/10
140/140 [==============================] - 22s 158ms/step - loss: 0.1958 - accuracy: 0.9217 - val_loss: 0.2715 - val_accuracy: 0.8900
Epoch 3/10
140/140 [==============================] - 24s 172ms/step - loss: 0.1050 - accuracy: 0.9617 - val_loss: 0.3441 - val_accuracy: 0.8730
Epoch 4/10
140/140 [==============================] - 23s 163ms/step - loss: 0.0656 - accuracy: 0.9776 - val_loss: 0.4310 - val_accuracy: 0.8855
Epoch 5/10
140/140 [==============================] - 24s 174ms/step - loss: 0.0380 - accuracy: 0.9877 - val_loss: 0.5228 - val_accuracy: 0.8748
Epoch 6/10
140/140 [==============================] - 24s 169ms/step - loss: 0.0313 - accuracy: 0.9868 - val_loss: 0.5554 - val_accuracy: 0.8730
Epoch 7/10
140/140 [==============================] - 24s 170ms/step - loss: 0.0243 - accuracy: 0.9915 -

AttributeError: ignored